# LightGCN V7 - Multi-Enhancement Version

## 개선 사항 (V6 대비)
1. **80/10/10 Split**: 더 많은 학습 데이터 (70% → 80%)
2. **Hard Negative Mining**: In-batch + Random negatives
3. **Larger Model**: emb_dim 64, layers 2
4. **Layer Normalization**: 학습 안정성
5. **More Negatives**: NUM_NEG 8

V6에서 유지:
- Confidence-Weighted BPR Loss
- Nuclear Norm Regularization
- Incoherence Penalty

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Device: {device} ({torch.cuda.get_device_name()})')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print(f'Device: {device}')
else:
    device = torch.device('cpu')
    print(f'Device: {device}')

Device: mps


## 1. 데이터 로딩 및 80/10/10 Split

In [2]:
df = pd.read_csv('../data/train.csv')

user2idx = {u: i for i, u in enumerate(sorted(df['user'].unique()))}
item2idx = {it: i for i, it in enumerate(sorted(df['item'].unique()))}
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {i: it for it, i in item2idx.items()}

n_users, n_items = len(user2idx), len(item2idx)

df['user_idx'] = df['user'].map(user2idx)
df['item_idx'] = df['item'].map(item2idx)
df['label'] = (df['rating'] >= 4.0).astype(int)

positive_df = df[df['label'] == 1].copy()

print(f"Total ratings: {len(df):,}")
print(f"Positive ratings (>= 4.0): {len(positive_df):,}")
print(f"Users: {n_users}, Items: {n_items}")

Total ratings: 105,139
Positive ratings (>= 4.0): 51,830
Users: 668, Items: 10321


In [3]:
# 80/10/10 Split (더 많은 학습 데이터)
train_data, val_data, test_data = [], [], []

for user_idx in range(n_users):
    user_pos = positive_df[positive_df['user_idx'] == user_idx]
    n_pos = len(user_pos)
    
    if n_pos >= 3:
        user_pos = user_pos.sample(frac=1, random_state=SEED).reset_index(drop=True)
        
        # 80 / 10 / 10 split
        train_end = int(0.8 * n_pos)
        val_end = int(0.9 * n_pos)
        
        train_end = max(1, train_end)
        val_end = max(train_end + 1, val_end)
        
        train_data.append(user_pos.iloc[:train_end])
        val_data.append(user_pos.iloc[train_end:val_end])
        test_data.append(user_pos.iloc[val_end:])
        
    elif n_pos == 2:
        user_pos = user_pos.sample(frac=1, random_state=SEED).reset_index(drop=True)
        train_data.append(user_pos.iloc[:1])
        val_data.append(user_pos.iloc[1:])
        
    elif n_pos == 1:
        train_data.append(user_pos)

train_df = pd.concat(train_data, ignore_index=True)
val_df = pd.concat(val_data, ignore_index=True)
test_df = pd.concat(test_data, ignore_index=True)

print(f"80/10/10 Split:")
print(f"  Train: {len(train_df):,} ({100*len(train_df)/len(positive_df):.1f}%)")
print(f"  Valid: {len(val_df):,} ({100*len(val_df)/len(positive_df):.1f}%)")
print(f"  Test:  {len(test_df):,} ({100*len(test_df)/len(positive_df):.1f}%)")

80/10/10 Split:
  Train: 41,214 (79.5%)
  Valid: 5,140 (9.9%)
  Test:  5,476 (10.6%)


In [4]:
train_users = torch.LongTensor(train_df['user_idx'].values)
train_items = torch.LongTensor(train_df['item_idx'].values)
val_users = torch.LongTensor(val_df['user_idx'].values)
val_items = torch.LongTensor(val_df['item_idx'].values)
test_users = torch.LongTensor(test_df['user_idx'].values)
test_items = torch.LongTensor(test_df['item_idx'].values)

user_pos_items_train = defaultdict(set)
for u, i in zip(train_df['user_idx'].values, train_df['item_idx'].values):
    user_pos_items_train[int(u)].add(int(i))

user_neg_candidates = {}
for u in range(n_users):
    pos = user_pos_items_train[u]
    user_neg_candidates[u] = np.array(list(set(range(n_items)) - pos))

print(f"Pre-computed negative candidates")

Pre-computed negative candidates


## 2. Graph 구성

In [5]:
def build_graph(train_df):
    users = train_df['user_idx'].values
    items = train_df['item_idx'].values
    
    edge_u2i = np.array([users, items + n_users])
    edge_i2u = np.array([items + n_users, users])
    edge_index = torch.LongTensor(np.concatenate([edge_u2i, edge_i2u], axis=1))
    
    num_nodes = n_users + n_items
    deg = torch.zeros(num_nodes).scatter_add(0, edge_index[0], torch.ones(edge_index.shape[1]))
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    edge_weight = deg_inv_sqrt[edge_index[0]] * deg_inv_sqrt[edge_index[1]]
    
    return edge_index.to(device), edge_weight.to(device)

edge_index, edge_weight = build_graph(train_df)
print(f"Graph: {edge_index.shape[1]:,} edges")

Graph: 82,428 edges


## 3. Enhanced LightGCN with Layer Normalization

In [6]:
class EnhancedLightGCN(nn.Module):
    """
    LightGCN with:
    - Layer Normalization for stability
    - Nuclear Norm Regularization
    - Incoherence Penalty
    """
    def __init__(self, n_users, n_items, emb_dim=64, n_layers=2):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.n_layers = n_layers
        
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
        
        # Layer Normalization (optional, per layer)
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(emb_dim) for _ in range(n_layers)
        ])
    
    def forward(self, edge_index, edge_weight):
        all_emb = torch.cat([self.user_emb.weight, self.item_emb.weight], dim=0)
        embs = [all_emb]
        
        for layer_idx in range(self.n_layers):
            row, col = edge_index
            messages = all_emb[col] * edge_weight.unsqueeze(1)
            all_emb = torch.zeros_like(all_emb).scatter_add(
                0, row.unsqueeze(1).expand(-1, self.emb_dim), messages
            )
            # Layer Normalization
            all_emb = self.layer_norms[layer_idx](all_emb)
            embs.append(all_emb)
        
        final_emb = torch.mean(torch.stack(embs), dim=0)
        return final_emb[:self.n_users], final_emb[self.n_users:]
    
    def nuclear_norm_reg(self):
        u_norm = torch.norm(self.user_emb.weight, p='fro') ** 2
        v_norm = torch.norm(self.item_emb.weight, p='fro') ** 2
        return (u_norm + v_norm) / 2
    
    def incoherence_penalty(self):
        n_sample = min(100, self.n_users)
        idx = torch.randperm(self.n_users, device=self.user_emb.weight.device)[:n_sample]
        U_sample = self.user_emb.weight[idx]
        U_norm = F.normalize(U_sample, p=2, dim=1)
        gram = torch.mm(U_norm, U_norm.t())
        mask = 1 - torch.eye(n_sample, device=gram.device)
        off_diag = gram * mask
        penalty = torch.sum(off_diag ** 2) / (n_sample * (n_sample - 1))
        return penalty

In [7]:
# Enhanced Model Configuration
EMB_DIM = 32      # 32 → 64 (더 풍부한 표현)
N_LAYERS = 1      # 1 → 2 (더 깊은 propagation)

model = EnhancedLightGCN(n_users, n_items, EMB_DIM, N_LAYERS).to(device)

print(f"Model Configuration:")
print(f"  emb_dim: {EMB_DIM}")
print(f"  n_layers: {N_LAYERS}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

Model Configuration:
  emb_dim: 32
  n_layers: 1
  Parameters: 351,712


## 4. Hard Negative Mining + Confidence Weighting

In [8]:
# Confidence 계산 (V6과 동일)
user_activity = train_df.groupby('user_idx').size().values
item_popularity = train_df.groupby('item_idx').size().reindex(range(n_items), fill_value=0).values

user_confidence = np.log1p(user_activity)
item_confidence = np.log1p(item_popularity)

user_confidence = 0.5 + (user_confidence - user_confidence.min()) / (user_confidence.max() - user_confidence.min())
item_confidence = 0.5 + (item_confidence - item_confidence.min()) / (item_confidence.max() - item_confidence.min() + 1e-8)

user_conf_tensor = torch.FloatTensor(user_confidence).to(device)
item_conf_tensor = torch.FloatTensor(item_confidence).to(device)

print(f"Confidence weights computed")

Confidence weights computed


In [9]:
def hard_negative_sampling(pos_users, pos_items, num_neg=8):
    """
    Hard Negative Mining:
    - 50% in-batch negatives (다른 user의 positive items)
    - 50% random negatives
    """
    batch_size = len(pos_users)
    neg_items = np.zeros((batch_size, num_neg), dtype=np.int64)
    
    n_inbatch = num_neg // 2
    n_random = num_neg - n_inbatch
    
    pos_items_np = pos_items.cpu().numpy() if torch.is_tensor(pos_items) else pos_items
    pos_users_np = pos_users.cpu().numpy() if torch.is_tensor(pos_users) else pos_users
    
    for i in range(batch_size):
        user_idx = int(pos_users_np[i])
        user_pos_set = user_pos_items_train[user_idx]
        
        # In-batch negatives
        in_batch_candidates = []
        for j in range(batch_size):
            if j != i:
                item_j = int(pos_items_np[j])
                if item_j not in user_pos_set:
                    in_batch_candidates.append(item_j)
        
        if len(in_batch_candidates) >= n_inbatch:
            selected_inbatch = np.random.choice(in_batch_candidates, size=n_inbatch, replace=False)
        else:
            selected_inbatch = np.array(in_batch_candidates) if in_batch_candidates else np.array([], dtype=np.int64)
            n_random += (n_inbatch - len(selected_inbatch))
        
        # Random negatives
        cands = user_neg_candidates[user_idx]
        if len(selected_inbatch) > 0:
            cands = np.setdiff1d(cands, selected_inbatch)
        selected_random = np.random.choice(cands, size=min(n_random, len(cands)), replace=False)
        
        all_neg = np.concatenate([selected_inbatch, selected_random])
        
        while len(all_neg) < num_neg:
            extra = np.random.choice(user_neg_candidates[user_idx], size=1)
            all_neg = np.concatenate([all_neg, extra])
        
        neg_items[i] = all_neg[:num_neg]
    
    return torch.LongTensor(neg_items)

print("Hard Negative Mining ready")

Hard Negative Mining ready


In [10]:
def confidence_weighted_bpr_loss(pos_scores, neg_scores, user_ids, pos_item_ids):
    confidence = user_conf_tensor[user_ids] * item_conf_tensor[pos_item_ids]
    diff = pos_scores.unsqueeze(1) - neg_scores
    bpr = -torch.log(torch.sigmoid(diff) + 1e-8).mean(dim=1)
    weighted_loss = (bpr * confidence).sum() / confidence.sum()
    return weighted_loss

## 5. 평가 함수

In [11]:
@torch.no_grad()
def evaluate(model, eval_users, eval_items, k=10, n_neg=99, sample_size=None):
    model.eval()
    u_emb, i_emb = model(edge_index, edge_weight)
    
    if sample_size and len(eval_users) > sample_size:
        sample_idx = np.random.choice(len(eval_users), sample_size, replace=False)
    else:
        sample_idx = np.arange(len(eval_users))
    
    hits, ndcgs = [], []
    
    for idx in sample_idx:
        user_idx = int(eval_users[idx])
        pos_item = int(eval_items[idx])
        
        cands = user_neg_candidates[user_idx]
        if len(cands) < n_neg:
            continue
        
        neg_items = np.random.choice(cands, size=n_neg, replace=False)
        candidates = np.concatenate([[pos_item], neg_items])
        
        u_t = torch.full((len(candidates),), user_idx, dtype=torch.long, device=device)
        i_t = torch.LongTensor(candidates).to(device)
        scores = (u_emb[u_t] * i_emb[i_t]).sum(dim=1).cpu().numpy()
        
        rank = (scores > scores[0]).sum() + 1
        hits.append(1.0 if rank <= k else 0.0)
        ndcgs.append(1.0 / np.log2(rank + 1) if rank <= k else 0.0)
    
    return np.mean(hits), np.mean(ndcgs)

print("Evaluation ready")

Evaluation ready


## 6. 학습

In [12]:
# Hyperparameters
LR = 1e-3           # 5e-3 → 1e-3 (더 안정적)
WEIGHT_DECAY = 1e-5
EPOCHS = 50
BATCH_SIZE = 1024
NUM_NEG = 8         # 4 → 8 (더 많은 negative)

# Regularization
LAMBDA_NUCLEAR = 1e-6
LAMBDA_INCOH = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

history = {
    'loss': [], 'bpr_loss': [], 'nuclear_loss': [], 'incoh_loss': [],
    'val_hit@10': [], 'val_ndcg@10': []
}

best_val_hit = 0
best_val_ndcg = 0
n_train = len(train_users)

print(f"Training EnhancedLightGCN V7")
print(f"Config: emb={EMB_DIM}, layers={N_LAYERS}, BS={BATCH_SIZE}, LR={LR}, NEG={NUM_NEG}")
print(f"Regularization: Nuclear={LAMBDA_NUCLEAR}, Incoh={LAMBDA_INCOH}")
print(f"Features: Hard Negative Mining + Confidence Weighting + Layer Norm")
print("=" * 70)

start_time = time.time()

Training EnhancedLightGCN V7
Config: emb=32, layers=1, BS=1024, LR=0.001, NEG=8
Regularization: Nuclear=1e-06, Incoh=0.001
Features: Hard Negative Mining + Confidence Weighting + Layer Norm


In [13]:
for epoch in range(EPOCHS):
    model.train()
    perm = torch.randperm(n_train)
    
    epoch_loss = 0
    epoch_bpr = 0
    epoch_nuclear = 0
    epoch_incoh = 0
    n_batches = 0
    
    for i in range(0, n_train, BATCH_SIZE):
        batch_idx = perm[i:i+BATCH_SIZE]
        pos_u = train_users[batch_idx].to(device)
        pos_i = train_items[batch_idx].to(device)
        
        # Hard Negative Mining
        neg_i = hard_negative_sampling(pos_u, pos_i, NUM_NEG).to(device)
        
        # Forward
        u_emb, i_emb = model(edge_index, edge_weight)
        
        # Scores
        pos_scores = (u_emb[pos_u] * i_emb[pos_i]).sum(dim=1)
        neg_u_expanded = pos_u.unsqueeze(1).expand(-1, NUM_NEG)
        neg_scores = (u_emb[neg_u_expanded] * i_emb[neg_i]).sum(dim=2)
        
        # Losses
        bpr_loss = confidence_weighted_bpr_loss(pos_scores, neg_scores, pos_u, pos_i)
        nuclear_loss = LAMBDA_NUCLEAR * model.nuclear_norm_reg()
        incoh_loss = LAMBDA_INCOH * model.incoherence_penalty()
        
        total_loss = bpr_loss + nuclear_loss + incoh_loss
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        epoch_loss += total_loss.item()
        epoch_bpr += bpr_loss.item()
        epoch_nuclear += nuclear_loss.item()
        epoch_incoh += incoh_loss.item()
        n_batches += 1
    
    history['loss'].append(epoch_loss / n_batches)
    history['bpr_loss'].append(epoch_bpr / n_batches)
    history['nuclear_loss'].append(epoch_nuclear / n_batches)
    history['incoh_loss'].append(epoch_incoh / n_batches)
    
    if (epoch + 1) % 5 == 0:
        val_hit, val_ndcg = evaluate(model, val_users, val_items, sample_size=3000)
        history['val_hit@10'].append(val_hit)
        history['val_ndcg@10'].append(val_ndcg)
        
        print(f"Epoch {epoch+1:3d} | Loss: {epoch_loss/n_batches:.4f} "
              f"(BPR: {epoch_bpr/n_batches:.4f}) "
              f"| Val Hit@10: {val_hit:.4f} | Val NDCG@10: {val_ndcg:.4f}")
        
        if val_hit > best_val_hit:
            best_val_hit = val_hit
            best_val_ndcg = val_ndcg
            torch.save(model.state_dict(), 'best_lightgcn_v7.pt')

elapsed = time.time() - start_time
print(f"\nTraining completed in {elapsed/60:.1f} minutes")
print(f"Best Validation: Hit@10={best_val_hit:.4f}, NDCG@10={best_val_ndcg:.4f}")

Epoch   5 | Loss: 0.1089 (BPR: 0.1088) | Val Hit@10: 0.7220 | Val NDCG@10: 0.4705
Epoch  10 | Loss: 0.0701 (BPR: 0.0700) | Val Hit@10: 0.7127 | Val NDCG@10: 0.4593
Epoch  15 | Loss: 0.0540 (BPR: 0.0539) | Val Hit@10: 0.6937 | Val NDCG@10: 0.4498


KeyboardInterrupt: 

Epoch   5 | Loss: 0.1089 (BPR: 0.1088) | Val Hit@10: 0.7220 | Val NDCG@10: 0.4705
Epoch  10 | Loss: 0.0701 (BPR: 0.0700) | Val Hit@10: 0.7127 | Val NDCG@10: 0.4593
Epoch  15 | Loss: 0.0540 (BPR: 0.0539) | Val Hit@10: 0.6937 | Val NDCG@10: 0.4498
Epoch  20 | Loss: 0.0449 (BPR: 0.0448) | Val Hit@10: 0.7060 | Val NDCG@10: 0.4494
Epoch  25 | Loss: 0.0398 (BPR: 0.0397) | Val Hit@10: 0.6963 | Val NDCG@10: 0.4448
Epoch  30 | Loss: 0.0365 (BPR: 0.0364) | Val Hit@10: 0.6970 | Val NDCG@10: 0.4454
Epoch  35 | Loss: 0.0339 (BPR: 0.0338) | Val Hit@10: 0.6903 | Val NDCG@10: 0.4397
Epoch  40 | Loss: 0.0314 (BPR: 0.0313) | Val Hit@10: 0.6873 | Val NDCG@10: 0.4432
Epoch  45 | Loss: 0.0299 (BPR: 0.0298) | Val Hit@10: 0.6880 | Val NDCG@10: 0.4357
Epoch  50 | Loss: 0.0292 (BPR: 0.0290) | Val Hit@10: 0.6940 | Val NDCG@10: 0.4392

Training completed in 46.2 minutes
Best Validation: Hit@10=0.7220, NDCG@10=0.4705

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['loss'], 'b-', linewidth=2)
axes[0].set_title('Total Loss')
axes[0].set_xlabel('Epoch')
axes[0].grid(alpha=0.3)

epochs_val = np.arange(5, EPOCHS+1, 5)[:len(history['val_hit@10'])]
axes[1].plot(epochs_val, history['val_hit@10'], 'go-', linewidth=2)
axes[1].axhline(y=0.7821, color='red', linestyle='--', label='V6 Test')
axes[1].set_title('Validation Hit@10')
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(alpha=0.3)

axes[2].plot(epochs_val, history['val_ndcg@10'], 'ro-', linewidth=2)
axes[2].axhline(y=0.5217, color='blue', linestyle='--', label='V6 Test')
axes[2].set_title('Validation NDCG@10')
axes[2].set_xlabel('Epoch')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.suptitle('EnhancedLightGCN V7 Training', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Test Set 최종 평가

In [ ]:
model.load_state_dict(torch.load('best_lightgcn_v7.pt'))
model.eval()

print("=" * 70)
print("FINAL TEST SET EVALUATION")
print("=" * 70)

test_hit, test_ndcg = evaluate(model, test_users, test_items, sample_size=None)

print(f"\nTest Set Results:")
print(f"  Hit@10:  {test_hit:.4f}")
print(f"  NDCG@10: {test_ndcg:.4f}")

print(f"\nValidation Set Results:")
print(f"  Hit@10:  {best_val_hit:.4f}")
print(f"  NDCG@10: {best_val_ndcg:.4f}")

print(f"\nV6 → V7 Comparison:")
print(f"  V6 Test: Hit@10=0.7821, NDCG@10=0.5217")
print(f"  V7 Test: Hit@10={test_hit:.4f}, NDCG@10={test_ndcg:.4f}")
print(f"  Δ Hit@10:  {100*(test_hit - 0.7821)/0.7821:+.2f}%")
print(f"  Δ NDCG@10: {100*(test_ndcg - 0.5217)/0.5217:+.2f}%")

## 8. Threshold & O/X 추론

In [ ]:
with torch.no_grad():
    u_emb, i_emb = model(edge_index, edge_weight)

val_pos_scores = (u_emb[val_users.to(device)] * i_emb[val_items.to(device)]).sum(dim=1).cpu().numpy()

val_neg_scores = []
for user_idx in val_df['user_idx'].unique():
    n_pos = (val_df['user_idx'] == user_idx).sum()
    cands = user_neg_candidates[int(user_idx)]
    neg_items = np.random.choice(cands, size=min(n_pos, len(cands)), replace=False)
    
    u_t = torch.full((len(neg_items),), user_idx, dtype=torch.long, device=device)
    i_t = torch.LongTensor(neg_items).to(device)
    scores = (u_emb[u_t] * i_emb[i_t]).sum(dim=1).cpu().numpy()
    val_neg_scores.extend(scores)

all_scores = np.concatenate([val_pos_scores, np.array(val_neg_scores)])
all_labels = np.concatenate([np.ones(len(val_pos_scores)), np.zeros(len(val_neg_scores))])

print(f"Threshold tuning on {len(all_scores)} samples")

In [ ]:
thresholds = np.percentile(all_scores, [30, 40, 50, 60, 70, 80, 90])

print("Threshold Tuning:")
print(f"{'Threshold':<12} {'Precision':<12} {'Recall':<12} {'F1':<12}")
print("-" * 50)

best_prec, best_th, best_f1 = 0, 0, 0
for th in thresholds:
    preds = (all_scores >= th).astype(int)
    tp = ((preds == 1) & (all_labels == 1)).sum()
    fp = ((preds == 1) & (all_labels == 0)).sum()
    fn = ((preds == 0) & (all_labels == 1)).sum()
    
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    
    print(f"{th:<12.4f} {prec:<12.4f} {rec:<12.4f} {f1:<12.4f}")
    
    if prec >= 0.6 and prec > best_prec:
        best_prec, best_th, best_f1 = prec, th, f1
    elif best_prec < 0.6 and f1 > best_f1:
        best_f1, best_th, best_prec = f1, th, prec

print("-" * 50)
print(f"Selected: {best_th:.4f} (Precision: {best_prec:.4f})")

In [ ]:
def predict_ox(test_pairs_df):
    results = []
    
    known_mask = test_pairs_df['user'].isin(user2idx) & test_pairs_df['item'].isin(item2idx)
    known_df = test_pairs_df[known_mask]
    unknown_df = test_pairs_df[~known_mask]
    
    if len(known_df) > 0:
        u_idx = torch.LongTensor([user2idx[u] for u in known_df['user']]).to(device)
        i_idx = torch.LongTensor([item2idx[i] for i in known_df['item']]).to(device)
        
        with torch.no_grad():
            scores = (u_emb[u_idx] * i_emb[i_idx]).sum(dim=1).cpu().numpy()
        
        for (_, row), score in zip(known_df.iterrows(), scores):
            results.append({
                'user': row['user'], 'item': row['item'],
                'recommend': 'O' if score >= best_th else 'X'
            })
    
    for _, row in unknown_df.iterrows():
        results.append({'user': row['user'], 'item': row['item'], 'recommend': 'X'})
    
    return pd.DataFrame(results)

preds = predict_ox(val_df[['user', 'item']])
o_ratio = (preds['recommend'] == 'O').mean()
print(f"\nO ratio: {100*o_ratio:.1f}%")
print(preds.head(10).to_string(index=False))

## 9. 최종 요약

In [ ]:
print("=" * 70)
print("EnhancedLightGCN V7 Final Summary")
print("=" * 70)

print(f"\nEnhancements over V6:")
print(f"  1. Data Split: 70/15/15 → 80/10/10 (+10% training data)")
print(f"  2. Hard Negative Mining: In-batch + Random")
print(f"  3. Model Size: emb={EMB_DIM}, layers={N_LAYERS}")
print(f"  4. Layer Normalization: Stability")
print(f"  5. More Negatives: NUM_NEG={NUM_NEG}")

print(f"\nModel Configuration:")
print(f"  emb_dim: {EMB_DIM}")
print(f"  n_layers: {N_LAYERS}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

print(f"\nData Split:")
print(f"  Train: {len(train_df):,} ({100*len(train_df)/len(positive_df):.1f}%)")
print(f"  Valid: {len(val_df):,} ({100*len(val_df)/len(positive_df):.1f}%)")
print(f"  Test:  {len(test_df):,} ({100*len(test_df)/len(positive_df):.1f}%)")

print(f"\nPerformance:")
print(f"  Validation: Hit@10={best_val_hit:.4f}, NDCG@10={best_val_ndcg:.4f}")
print(f"  Test:       Hit@10={test_hit:.4f}, NDCG@10={test_ndcg:.4f}")
print(f"  Precision:  {best_prec:.4f}")
print(f"  O ratio:    {100*o_ratio:.1f}%")

In [ ]:
# Final test inference
# final_test_df = pd.read_csv('../data/test.csv')
# final_preds = predict_ox(final_test_df)
# final_preds.to_csv('predictions_v7.csv', index=False)

print("Ready for final test inference.")